# PDF RAG 에 대한 RAGAS 평가

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Project-RAG-With-Evaluation")

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## PDF RAG 체인

In [3]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 로드
pdf = PDFRetrievalChain(["data/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

# retriever 와 chain을 생성
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

## RAGAS 평가

ragas 평가를 위한 클래스를 정의합니다.

평가 metric 은 `answer_relevancy` 와 `faithfulness` 를 사용합니다.

In [4]:
from datasets import Dataset
from langchain_core.documents import Document
from ragas.metrics import answer_relevancy, faithfulness
from ragas import evaluate
from typing import List, Dict


class RagEvaluator:
    def __init__(self):
        # 데이터 저장을 위한 리스트 초기화
        self.questions: List[str] = []
        self.answers: List[str] = []
        self.contexts: List[List[Document]] = []

    def add_sample(self, question: str, answer: str, context: List[Document]):
        """평가할 데이터 샘플을 추가합니다."""
        self.questions.append(question)
        self.answers.append(answer)
        context_list = [doc.page_content for doc in context]
        self.contexts.append(context_list)

    def get_samples(self) -> Dict:
        """현재까지 저장된 모든 샘플을 딕셔너리 형태로 반환합니다."""
        return {
            "question": self.questions,
            "answer": self.answers,
            "contexts": self.contexts,
        }

    def evaluate_all(self):
        """저장된 데이터에 대해 RAG 평가를 수행합니다."""
        if not self.questions:
            raise ValueError(
                "평가할 데이터가 없습니다. add_sample()을 통해 데이터를 먼저 추가해주세요."
            )

        # Dataset 생성
        dataset = Dataset.from_dict(self.get_samples())

        # 평가 수행
        score = evaluate(dataset, metrics=[answer_relevancy, faithfulness])

        return score.to_pandas()

    def evaluate_last(self):
        """마지막 샘플에 대해 RAG 평가를 수행합니다."""
        if not self.questions:
            raise ValueError(
                "평가할 데이터가 없습니다. add_sample()을 통해 데이터를 먼저 추가해주세요."
            )

        last_sample = {
            "question": [self.get_samples()["question"][-1]],
            "answer": [self.get_samples()["answer"][-1]],
            "contexts": [self.get_samples()["contexts"][-1]],
        }

        dataset = Dataset.from_dict(last_sample)
        score = evaluate(dataset, metrics=[answer_relevancy, faithfulness])
        return score.to_pandas()

    def clear(self):
        """평가 데이터를 초기화합니다."""
        self.questions = []
        self.answers = []
        self.contexts = []


# 사용 예시
evaluator = RagEvaluator()

## 실행

In [5]:
from langchain_teddynote.messages import stream_response


def ask(question: str):
    context = pdf_retriever.invoke(question)
    response = pdf_chain.stream(
        {
            "question": question,
            "context": context,
        }
    )
    output = stream_response(response, return_output=True)
    evaluator.add_sample(question, output, context)

In [6]:
# 질문 실행
ask("삼성전자가 만든 생성형 AI 모델은 무엇인가요?")

삼성전자가 만든 생성형 AI 모델은 '삼성 가우스'입니다.

**Source**
- data/SPRI_AI_Brief_2023년12월호_F.pdf (page 12)

In [7]:
# 질문 실행
ask("구글이 anthropic 에 투자한 금액은 얼마인가요?")

구글이 앤스로픽에 투자한 금액은 최대 20억 달러입니다. 이 중 5억 달러를 우선 투자하고, 향후 15억 달러를 추가로 투자할 계획입니다.

**Source**
- data/SPRI_AI_Brief_2023년12월호_F.pdf (page 13)

In [8]:
# 샘플을 출력
evaluator.get_samples()

{'question': ['삼성전자가 만든 생성형 AI 모델은 무엇인가요?', '구글이 anthropic 에 투자한 금액은 얼마인가요?'],
 'answer': ["삼성전자가 만든 생성형 AI 모델은 '삼성 가우스'입니다.\n\n**Source**\n- data/SPRI_AI_Brief_2023년12월호_F.pdf (page 12)",
  '구글이 앤스로픽에 투자한 금액은 최대 20억 달러입니다. 이 중 5억 달러를 우선 투자하고, 향후 15억 달러를 추가로 투자할 계획입니다.\n\n**Source**\n- data/SPRI_AI_Brief_2023년12월호_F.pdf (page 13)'],
 'contexts': [['SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nKEY Contents\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성\nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한\n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원',
   '£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델\n‘삼성 가우스’를 최초 공개\n∙ 정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에\n최적화된 크기의 모델 선택이 가능\n∙ 삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,\n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유',
   '▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ·············

단일 평가를 진행합니다.

In [9]:
# 평가 결과 출력
evaluate_last = evaluator.evaluate_last()
evaluate_last

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,answer_relevancy,faithfulness
0,구글이 anthropic 에 투자한 금액은 얼마인가요?,[기업 허깅 페이스(Hugging Face)에도 투자\n∙ 구글은 챗GPT의 기반 ...,구글이 앤스로픽에 투자한 금액은 최대 20억 달러입니다. 이 중 5억 달러를 우선 ...,0.892348,1.0


In [10]:
print(
    f'✅ 평가 결과\n- 관련성 점수: {evaluate_last.iloc[0]["answer_relevancy"]:.3f}\n- 신뢰도 점수: {evaluate_last.iloc[0]["faithfulness"]:.3f}'
)

✅ 평가 결과
- 관련성 점수: 0.892
- 신뢰도 점수: 1.000


종합 평가를 진행합니다.

In [11]:
# 평가 결과 출력
evaluator.evaluate_all()

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,answer_relevancy,faithfulness
0,삼성전자가 만든 생성형 AI 모델은 무엇인가요?,"[SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 AI...",삼성전자가 만든 생성형 AI 모델은 '삼성 가우스'입니다.\n\n**Source**...,0.906562,1.0
1,구글이 anthropic 에 투자한 금액은 얼마인가요?,[기업 허깅 페이스(Hugging Face)에도 투자\n∙ 구글은 챗GPT의 기반 ...,구글이 앤스로픽에 투자한 금액은 최대 20억 달러입니다. 이 중 5억 달러를 우선 ...,0.892348,1.0
